## Acessing sql database to get tweets

In [3]:
import pandas as pd
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../data/FIFA 22")
df = pd.read_sql_query("SELECT user, full_text  FROM tweets", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

                  user                                          full_text
0   762319514734297088  2nd 125k #EASPORTSFIFA20 @Castro1021 @TwoSyncO...
1   859350438461685760  🌐TTMWW🌐▪️ #FIFA22 is here #Oct. 1, 2021. The #...
2  1349475286451105798  THREAD : Idées de concepts pour le mode carriè...
3  1016289053438828545  Je viens d'acheter fifa 22 mais y a pas un pti...
4             61085376  AGARRÓN INTERNACIONAL 👊🏼🔥\n\nFunes Mori - Luka...


In [5]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")


def process_text(text):
    text = re.sub(r'http\S+', '', text)
    text = hashtags.sub(' hashtag', text)
    text = mentions.sub(' entity', text)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())


df["Test"] = df.full_text.apply(process_text)

In [6]:
df.drop(columns='full_text', inplace=True)

In [8]:
import numpy as np

df['Test'].replace('', np.nan, inplace=True)

In [9]:
df.dropna(subset=['Test'], inplace=True)

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("../models/trial5")
model = AutoModelForSequenceClassification.from_pretrained("../models/trial5")

In [14]:
import torch


def predictionPipeline(text):
    #id_tolabel = {0: "Negative", 2: "Positive"}
    modeleval = model.eval()
    tokenized = tokenizer(text, return_tensors='pt', truncation=True).to(modeleval.device)
    with torch.no_grad(): label = torch.argmax(model.forward(**tokenized).logits, dim=1)[0].cuda().item()
    return label


df['prediction'] = df['Test'].apply(predictionPipeline)

In [15]:
df_positive = df[df['prediction'] == 2]

In [ ]:
add_handler(df_positive['user'])
add_liked_games(df_positive['user'], game_name)

In [45]:
score = df[df["prediction"] == 2].count()[0] - df[df["prediction"] == 0].count()[0]